In [ ]:
import sys, dill, pickle, pandas, re, HTSeq, os, importlib, operator, functools, random, matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from matplotlib import patches

import sklearn
from sklearn.neighbors.kde import KernelDensity

top_dir = top = '/Users/dp/pma/dataAndScripts/clip/meta/'
os.chdir(top_dir)

sys.path.append('/Users/dp/pma/')
import sameRiver
from sameRiver import *

import sameRiver.peakFinder
import sameRiver.peakSeqFinder

importlib.reload(sameRiver.peakFinder)
importlib.reload(sameRiver.peakSeqFinder)

figname = lambda _str: f'/Users/dp/pma/dataAndScripts/clip/figs/{_str}.pdf'

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rc('font',**{'family':'sans-serif','sans-serif':['Arial']})

In [ ]:
import sameRiver.peakArtist
import sameRiver.rnaDataFileMaker

maker = sameRiver.rnaDataFileMaker.rnaDataFileMaker()
RNAs = maker.make_from_gtf_file(
    gtf_filename='/Users/dp/pma/repeats_and_genome.gtf',  # New mapping method gtf.
    #gtf_filename='/opt/genomes/test.gtf',
    #gtf_filename='/opt/genomes/repeats_and_ensembl_release94_GRCh38/combined_tsl1andNA.gtf',  # Old mapping gtf.
    output_data_filename=f"{top}/data/rna.data",
)



In [ ]:
importlib.reload(sameRiver.peakArtist)
pk = sameRiver.peakArtist.peakArtist(
    scheme_filename=f'{top}/scheme.xlsx',
    bedgraph_folder=f'{top}/beds/read_start/',
    file_with_total_read_counts=f'{top}/data/total_read_numbers.txt',
    rnas_obj=RNAs)

In [ ]:
import sameRiver.artist
from typing import List, Mapping, Union
importlib.reload(sameRiver.artist)
import glob
from sameRiver.artist import draw_gene_body, draw_ivs, array_from_htseq, signal_for_protein_in_region, smooth_y_values

def drawNlines(_x: np.array, y_arrays: List[np.array]=[], labels: list=[], title: str='') -> None:
    linewidth=1#0.5
    a = 1

    pal = sns.cubehelix_palette(len(labels), start=0.14, rot=0.9)
    for n, (label, y_array) in enumerate(zip(labels, y_arrays)):
        plt.plot(
            _x, y_array, label=label, c=pal[n], linewidth=linewidth, alpha=a)

def plotN(
    _x: np.array, y_arrays: List[np.array]=[], labels: list=[], 
    title: str='', rna='rna', smooth=False, yaxislabel=''):
    
    plt.clf()
    fig = plt.figure()
    
    if smooth:
        xys = [smooth_y_values(y) for y in y_arrays]
        _x = xys[0][0]
        y_arrays = [_[1] for _ in xys]

    drawNlines(_x, y_arrays, labels=labels)

    plt.legend()#.remove()
    plt.title(title)
    plt.ylabel(yaxislabel)
    fig.set_figheight(4)  # 2 for publication.
    fig.set_figwidth(6)  # 3 for publication.
    fig.savefig(figname(f'signal_smooth_{title}'))
    plt.show()
    plt.clf()
    plt.close()

print(pk.gene_name_to_bedgraph_filenames('RPL5:E82K', glob.glob(pk.bedgraph_folder + '/*_+.wig')))
rna = '>0_5S#rRNA'
rna = '>0_rRNA_Hsa#rRNA'
rna = '>0_U2#snRNA'
rna = 'PURPL'
q = pk.genomic_iv_of_gene(rna)
print(q)
labels = ['RPL5', 'FUBP1', 'PCBP1', 'SF3B1']
labels = ['SF3B1', 'FUBP1', 'KHDRBS2', 'KHDRBS2:R168C', 'A1CF',  'A1CF:E34K']
xy_arrs = [pk.signal_for_protein_in_region(label, rna) for label in labels]
print(xy_arrs[0])
#xy_arrs = [(_x[10:], _y[10:]) for _x,_y in xy_arrs]
#print(xy_arrs)
x_arrs = [_[0] for _ in xy_arrs]
y_arrs = [_[1] for _ in xy_arrs]
print(x_arrs)
plotN(x_arrs[0], y_arrs, labels=labels, title=rna, smooth=True)


In [ ]:
import joypy

xys = [smooth_y_values(y) for y in y_arrs]
smoothed_ys = [_[1] for _ in xys]
a = np.array(smoothed_ys)
df = pandas.DataFrame(a.T, columns=labels)

fig = plt.figure()
j = joypy.joyplot(df, kind='values', #ylim='own', 
              linecolor="k", fill=False, x_range=range(len(df.index)))
plt.savefig(figname('ridgeline_plot_PURPL_per_read'))
plt.show(); plt.clf(); plt.close()

In [ ]:
def to_per_protein(_df):
    for protein in _df.columns:
        xl_rate = pk.scheme.percent_xl_of_protein_from_file(protein)
        if xl_rate == 0:
            xl_rate = pk.scheme.percent_xl_of_protein_from_file(protein.replace(':', ' '))
        print(f"{protein}: {xl_rate}")
        _df[protein] = 1E4 * xl_rate * _df[protein]
    return _df

xldf = df.copy()

q = to_per_protein(xldf)

fig = plt.figure()
j = joypy.joyplot(q, kind='values', #ylim='own', 
              linecolor="k", fill=False, x_range=range(len(df.index)))
plt.savefig(figname('ridgeline_plot_PURPL_per_protein'))
plt.show(); plt.clf(); plt.close()

In [ ]:
import sameRiver.artist
importlib.reload(sameRiver.artist)
from sameRiver.artist import draw_gene_body, draw_ivs, array_from_htseq, signal_for_protein_in_region, smooth_y_values

##################
# Definitions

def draw3lines(_x, y_wt, y_100p, y_100q, title=''):
    linewidth=1#0.5
    a = 1

    plt.plot(_x, y_wt, label='WT', c='#142C37', linewidth=linewidth, alpha=a)
    plt.plot(_x, y_100p, label='L100P', c='#A5689A', linewidth=linewidth, alpha=a)
    plt.plot(_x, y_100q, label='L100Q', c='#899FC9', linewidth=linewidth, alpha=a)
    
def plot3(_x, y_wt, y_100p, y_100q, title='', rna='rna', ylabel='', smooth=False):
    plt.clf()
    fig = plt.figure()

    
    if smooth:
        (_x, sy_wt) = smooth_y_values(y_wt)
        (_x, sy_100p) = smooth_y_values(y_100p)
        (_x, sy_100q) = smooth_y_values(y_100q)
        draw3lines(_x, sy_wt, sy_100p, sy_100q)
    else:
        draw3lines(_x, y_wt, y_100p, y_100q)

    plt.legend().remove()
    plt.title(title)
    plt.ylabel(ylabel)
    fig.set_figheight(2)  # 2 for publication.
    fig.set_figwidth(3)  # 3 for publication.
    fig.savefig(
        '/Users/dp/pma/dataAndScripts/clip/figs/signal_smooth_{}_{}.pdf'.format(title, ylabel))
    plt.show()
    plt.clf()
    plt.close()

    
def iv_str(_str, strand):
    s = _str.split(':')
    a, b = [int(x.rstrip('\n')) for x in s[1].split('-')]
    return HTSeq.GenomicInterval(s[0], a, b, strand)

##################
# Input selections

"""chr11:65892136-65900526

FOSL1
chr11:65892136-65900526
id = NM_001300844
--------------
Exon number: 3
Amino acid coding number: 219
chr11:65892136-65893296
~
U2:
repeats:115971-116159



"""
#rna = 'RNU4-2'
#rna = 'GAPDH'
#rna = 'FOSL1'
rna = '>0_U1#snRNA'
#rna = '>0_U2#snRNA'
#rna = '>0_U3#snRNA'
#rna = '>0_U4#snRNA'
#rna = '>0_rRNA_Hsa#rRNA'
#rna = '>0_5S#rRNA'
#rna = 'CDKN1A'
#rna = 'SOX9'
#rna = 'PCBP2'
#rna = 'SMAD3'
#rna = 'SPRYD4'
#rna = 'AGPAT1'
#rna = 'GSK3A'
#rna = 'CDKN1A'

#rna = iv_str('chr11:65892136-65893296', '-')  # FOSL1 leftmost exon
#rna = iv_str('repeats:115971-116159', '-') # U2
#rna = iv_str('repeats:1217629-1217846', '-') # U3
#rna = iv_str('repeats:10075-10220', '-')
#rna = iv_str('chr19:42230186-42230867', '-')  # Last GSK3A exon

##################
# Obtaining arrays of values at selection

# Per million reads
#pk.signal_for_protein_in_region('RPL5', '>0_5S#rRNA')
_x, y_wt  = pk.signal_for_protein_in_region('PCBP1', rna)
_x, y_100p = pk.signal_for_protein_in_region('PCBP1:100P', rna)
_x, y_100q = pk.signal_for_protein_in_region('PCBP1:100Q', rna)
#y_wt, _x = signal_for_protein_in_region('PCBP1', ssr, rna, bedfname_to_total_reads)
#y_100p, _x = signal_for_protein_in_region('PCBP1:100P', ssr, rna, bedfname_to_total_reads)
#y_100q, _x = signal_for_protein_in_region('PCBP1:100Q', ssr, rna, bedfname_to_total_reads)
#y_wt, _ = signal_for_protein_in_region('PCBP1', ssr05, rna, bedfname_to_total_reads)

##################
# Derivatives of selections
scheme = sameRiver.scheme.scheme('./scheme.xlsx')
# Per billion proteins
y_wt_per_prot = 10 * scheme.percent_xl_of_protein_from_file('PCBP1') * y_wt
y_100p_per_prot = 10 * scheme.percent_xl_of_protein_from_file('PCBP1 L100P') * y_100p
y_100q_per_prot = 10 * scheme.percent_xl_of_protein_from_file('PCBP1 L100Q') * y_100q


# Per cell
df = pandas.read_excel('/Users/dp/pma/dataAndScripts/tables/lysateExpression.xlsx')

to_expression = dict(df.groupby('Protein')['Value'].mean())

y_100p_per_cell = 1E-2 * to_expression['PCBP1 100P'] * y_100p_per_prot
y_100q_per_cell = 1E-2 * to_expression['PCBP1 100Q'] * y_100q_per_prot
y_wt_per_cell = 1E-2 * to_expression['PCBP1'] * y_wt_per_prot

##################
# Plot.

if type(rna) != type(''):
    title = '{}_{}_{}'.format(rna.chrom, rna.start, rna.end)
else:
    title = rna

smooth = True

plot3(
    _x, y_wt, y_100p, y_100q, 
    rna=rna, title=title, ylabel='Cross-links per million cross-links', smooth=smooth)

plot3(
    _x, y_wt_per_prot, y_100p_per_prot, y_100q_per_prot,
    rna=rna, title=title, ylabel='Cross-links per billion proteins', smooth=smooth)

plot3(
    _x, y_wt_per_cell, y_100p_per_cell, y_100q_per_cell, 
    rna=rna, title=title, ylabel='Cross-links per cell (relative)', smooth=smooth)




In [ ]:
iv = HTSeq.GenomicInterval('a',1,20,'-')
print(type(iv.strand))
dict([('+', '-'), ('-', '+')])[str(iv.strand)]

In [ ]:
importlib.reload(sameRiver.statsForCountsNB)

#nb = sameRiver.statsForCountsNB.statsForCountsNB(negatives=negatives, positives=positives)
#nb.calculate_pvalues()
nb.write_pvals_single_file(
    which='per_read', outfname='data/pvals_for_signal_at_{}_peaks_per_read.xlsx'.format(gene))

In [ ]:
import scipy.stats as stats
import statsmodels
import statsmodels.api as sm

detected = np.array([1, 1, 2, 1, 2, 3, 5, 8]) * 100
average = 10 * 100
#detected = np.log(detected)
#average = np.log(average)

mean_negative = np.average(detected)
max_negative = np.max(detected)
vmr = (np.std(detected)**2)/mean_negative

pois = stats.poisson(mean_negative)
p = 1 - pois.cdf(average)

print(detected, 'vs', average)
print('mu=', mean_negative)
print('std=', np.std(detected))
print('VMR=', vmr)

q = sm.NegativeBinomial(detected, np.array([1] * len(detected)), loglike_method='nb2')
res = q.fit(disp=0)
mu = res.predict()[0]
size = 1. / res.params[1]
prob = size / (size + mu)
print('NB fit mu=', mu)

dist_est = stats.nbinom(size, prob)

nb_p = 1 - dist_est.cdf(average)
print('Poisson p value=', p)
print('NB pvalue=', nb_p)

In [ ]:
#genomic_fasta_fname = '/opt/genome/repeats_and_ensembl_release94_GRCh38/combined.fa'
#genomic_fasta = dict( (s[1], s[0]) for s in HTSeq.FastaReader(genomic_fasta_fname, raw_iterator=True) )

motif = 'GCATG'

import sameRiver.motifTrackWriter
importlib.reload(sameRiver.motifTrackWriter)
from sameRiver.motifTrackWriter import motifTrackWriter
        
mtw = motifTrackWriter()
mtw.write_motif_track(genomic_fasta, motif, motif2='TGCATG')